In [1]:
import os, sys
from glob import glob
import numpy as np
import dask
import xarray as xr
import xgcm
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *
from mitequinox.sigp import *
from mitequinox.plot import *

In [2]:
from dask_jobqueue import PBSCluster
#cluster = PBSCluster(cores=12, walltime='03:00:00')
cluster = PBSCluster(cores=6, processes=6,  walltime='04:00:00')
w = cluster.scale(12*10)

In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [33]:
client

Client Scheduler: tcp://10.135.39.22:32915 Dashboard: http://10.135.39.22:8787/status,Cluster Workers: 120 Cores: 120 Memory: 2.00 TB


_______
# Useful information

In [5]:
E_dir = '/work/ALT/swot/aval/syn/xy/momentum_balance/hourly/'
time_length = 240

# define (real) time
def iters_to_date(iters, delta_t=3600.):
    t0 = datetime.datetime(2011,11,23,8)    
    ltime = delta_t * (np.array(iters))
    dtime = [t0+dateutil.relativedelta.relativedelta(seconds=t) for t in ltime]    
    return dtime

time_day = iters_to_date(np.arange(time_length))

grd = load_grd().reset_coords()
mask = ((grd.hFacW.rename({'i_g': 'i'}) == 1) &
        (grd.hFacS.rename({'j_g': 'j'}) == 1) 
       ).rename('mask').reset_coords(drop=True)
grd_rspec = xr.merge([mask, grd])

___________
# Term 1: time acceleration

### du/dt and dv/dt

In [6]:
dij = 4

dsU = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSU')).isel(time=slice(200,200+time_length))
dsV = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSV')).isel(time=slice(200,200+time_length))

ds_raw = (xr.merge([dsU,dsV]).assign_coords(**grd.variables))
ds_raw = ds_raw.assign_coords(time=time_day) 
grid = xgcm.Grid(ds_raw, periodic=['X', 'Y'])
print(ds_raw)
print('\n data size: %.1f GB' %(ds_raw.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 240)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    CS       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    Depth    (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    PHrefC   (k) float32 dask.array<chunksize=(90,), meta=np.ndarray>
    PHrefF 

In [7]:
for face in range(ds_raw['face'].size):
    
    d_SSU_dt_time = ( ds_raw['SSU'].isel(face=face).shift(time=-1) - ds_raw['SSU'].isel(face=face).shift(time=1) )/2/3600
    d_SSV_dt_time = ( ds_raw['SSV'].isel(face=face).shift(time=-1) - ds_raw['SSV'].isel(face=face).shift(time=1) )/2/3600
 
    dudt = grid.interp(d_SSU_dt_time,'X')
    dvdt = grid.interp(d_SSV_dt_time,'Y')    

    du_dt =  dudt.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds_raw.CS.isel(face=face, i=slice(0,None,dij), j=slice(0,None,dij)) - dvdt.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds_raw.SN.isel(face=face, i=slice(0,None,dij), j=slice(0,None,dij))
    dv_dt =  dudt.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds_raw.SN.isel(face=face, i=slice(0,None,dij), j=slice(0,None,dij)) + dvdt.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds_raw.CS.isel(face=face, i=slice(0,None,dij), j=slice(0,None,dij))    
    
    # store
    du_dt = np.real(du_dt).rename('du_dt')    
    du_dt = du_dt.chunk({'time': 1,'i':1080,'j':1080})
    du_dt.to_dataset().to_zarr(E_dir+'du_dt_f%02d.zarr'% (face), mode='w')

    dv_dt = np.real(dv_dt).rename('dv_dt')    
    dv_dt = dv_dt.chunk({'time': 1,'i':1080,'j':1080})
    dv_dt.to_dataset().to_zarr(E_dir+'dv_dt_f%02d.zarr'% (face), mode='w')        

    print('face=%d - allready processed'%(face))    

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


# A key term zeta

In [8]:
# No need to do these for KE, but it's necessary to do for zeta (i.e. no need to ratote uv first to compute zeta)
# 240 time_length about 50 mins

dij=4

for face in range(13):
#for face in [1]:

    zeta_3 = (grid.diff(ds_raw.dyC.isel(face=face) * ds_raw.SSV.isel(face=face),'X') - grid.diff(ds_raw.dxC.isel(face=face) * ds_raw.SSU.isel(face=face),'Y'))/ds_raw.rAz.isel(face=face) # relative vorticity  

    zeta = grid.interp(grid.interp(zeta_3,'Y'),'X').isel(i=slice(0,None,dij), j=slice(0,None,dij))   #(zeta) at i,j
    zeta = np.real(zeta).rename('zeta')
    zeta = zeta.chunk({'time': 1,'i':1080,'j':1080})
    %time zeta.to_dataset().to_zarr(E_dir+'zeta_rotation_f%02d.zarr'% (face), mode='w')

    print('face=%d - allready processed'%(face))

/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs


CPU times: user 3min 7s, sys: 7.17 s, total: 3min 14s
Wall time: 3min 10s
face=0 - allready processed


/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10

CPU times: user 3min 11s, sys: 8.22 s, total: 3min 19s
Wall time: 3min 16s
face=1 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10

...

CPU times: user 3min 16s, sys: 7.17 s, total: 3min 23s
Wall time: 3min 19s
face=10 - allready processed


/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10

CPU times: user 3min 17s, sys: 7.31 s, total: 3min 24s
Wall time: 3min 21s
face=11 - allready processed


/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 20
  **blockwise_kwargs
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10

CPU times: user 3min 16s, sys: 7.24 s, total: 3min 23s
Wall time: 3min 19s
face=12 - allready processed


___________
# Useful info.

In [9]:
grd = load_grd().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])
dsU = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSU')).isel(time=slice(200,200+time_length))
dsV = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSV')).isel(time=slice(200,200+time_length))
dsE = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('Eta')).isel(time=slice(1512+200,1512+200+time_length))
dsU

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


<xarray.Dataset>
Dimensions:  (face: 13, i_g: 4320, j: 4320, time: 240)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) float64 6.422e+06 6.426e+06 6.43e+06 ... 7.279e+06 7.283e+06
Data variables:
    SSU      (time, face, j, i_g) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>

In [10]:
# define (real) time
def iters_to_date(iters, delta_t=3600.):
    t0 = datetime.datetime(2011,11,23,8)    
    ltime = delta_t * (np.array(iters))
    dtime = [t0+dateutil.relativedelta.relativedelta(seconds=t) for t in ltime]    
    return dtime

time_day = iters_to_date(np.arange(time_length))

dsE = dsE.assign_coords(time=time_day) 
dsU = dsU.assign_coords(time=time_day) 
dsV = dsV.assign_coords(time=time_day) 
dsU = dsU.assign_coords(**grd.variables)
grid = xgcm.Grid(dsU, periodic=['X', 'Y'])
print(grid)

<xgcm.Grid>
X Axis (periodic):
  * center   i --> left
  * left     i_g --> center
Z Axis (not periodic):
  * center   k --> left
  * left     k_l --> center
  * outer    k_p1 --> center
  * right    k_u --> center
Y Axis (periodic):
  * center   j --> left
  * left     j_g --> center


In [11]:
for face in range(dsE['face'].size):
    
    Eta = dsE['Eta'].isel(face=face)   
    Eta = np.real(Eta).rename('Eta')
    Eta = Eta.chunk({'time': 1,'i':4320,'j':4320})
    %time Eta.to_dataset().to_zarr(E_dir+'Eta_f%02d.zarr'% (face), mode='w')
    
    print('face=%d - allready processed'%(face))

CPU times: user 2.84 s, sys: 223 ms, total: 3.06 s
Wall time: 3.75 s
face=0 - allready processed
CPU times: user 3 s, sys: 259 ms, total: 3.26 s
Wall time: 4.08 s
face=1 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


CPU times: user 4.01 s, sys: 291 ms, total: 4.31 s
Wall time: 5.43 s
face=2 - allready processed
CPU times: user 2.68 s, sys: 213 ms, total: 2.9 s
Wall time: 3.89 s
face=3 - allready processed
CPU times: user 3.19 s, sys: 251 ms, total: 3.44 s
Wall time: 4.19 s
face=4 - allready processed
CPU times: user 2.63 s, sys: 199 ms, total: 2.83 s
Wall time: 3.96 s
face=5 - allready processed
CPU times: user 3.08 s, sys: 273 ms, total: 3.35 s
Wall time: 4.23 s
face=6 - allready processed
CPU times: user 2.86 s, sys: 287 ms, total: 3.15 s
Wall time: 4.38 s
face=7 - allready processed
CPU times: user 3.18 s, sys: 244 ms, total: 3.43 s
Wall time: 4.71 s
face=8 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


CPU times: user 4.02 s, sys: 358 ms, total: 4.37 s
Wall time: 5.26 s
face=9 - allready processed
CPU times: user 2.79 s, sys: 224 ms, total: 3.02 s
Wall time: 3.85 s
face=10 - allready processed
CPU times: user 2.78 s, sys: 249 ms, total: 3.03 s
Wall time: 4.42 s
face=11 - allready processed
CPU times: user 2.93 s, sys: 255 ms, total: 3.18 s
Wall time: 4.02 s
face=12 - allready processed


In [12]:
for face in range(dsU['face'].size):
    
    U_ij = grid.interp(dsU['SSU'].isel(face=face),'X')
    V_ij = grid.interp(dsV['SSV'].isel(face=face),'Y')
    
    SSU_rotation = U_ij*dsU.CS.isel(face=face) - V_ij*dsU.SN.isel(face=face)
    SSV_rotation = U_ij*dsU.SN.isel(face=face) + V_ij*dsU.CS.isel(face=face)
    
    SSU_rotation = np.real(SSU_rotation).rename('SSU_rotation')
    SSU_rotation = SSU_rotation.chunk({'time': 1,'i':4320,'j':4320})
    SSU_rotation.to_dataset().to_zarr(E_dir+'SSU_rotation_f%02d.zarr'% (face), mode='w')

    SSV_rotation = np.real(SSV_rotation).rename('SSV_rotation')
    SSV_rotation = SSV_rotation.chunk({'time': 1,'i':4320,'j':4320})
    SSV_rotation.to_dataset().to_zarr(E_dir+'SSV_rotation_f%02d.zarr'% (face), mode='w')   
    
    print('face=%d - allready processed'%(face))

distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


face=0 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


face=1 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


...

face=4 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


face=5 - allready processed


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


## Read rotated uv

In [13]:
face_all = 13

D = [xr.open_zarr(E_dir+'SSU_rotation_f%02d.zarr'%(face)) for face in range(face_all)]
ds_SSU = xr.concat(D, dim='face')

D = [xr.open_zarr(E_dir+'SSV_rotation_f%02d.zarr'%(face)) for face in range(face_all)]
ds_SSV = xr.concat(D, dim='face')

D = [xr.open_zarr(E_dir+'Eta_f%02d.zarr'%(face)) for face in range(face_all)]
ds_Eta = xr.concat(D, dim='face')
%time ds = xr.merge([ds_SSU, ds_SSV, ds_Eta]).assign_coords(**grd.variables)
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

CPU times: user 1min 5s, sys: 22.6 s, total: 1min 27s
Wall time: 1min 33s
<xarray.Dataset>
Dimensions:       (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 240)
Coordinates:
  * i             (i) int64 0 1 2 3 4 5 6 ... 4313 4314 4315 4316 4317 4318 4319
  * j             (j) int64 0 1 2 3 4 5 6 ... 4313 4314 4315 4316 4317 4318 4319
  * time          (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
    CS            (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    Depth         (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    SN            (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    XC            (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    YC            (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
  * face          (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    i

In [14]:
grid = xgcm.Grid(grd, periodic=['X', 'Y'])
grid

<xgcm.Grid>
X Axis (periodic):
  * center   i --> left
  * left     i_g --> center
Z Axis (not periodic):
  * center   k --> left
  * left     k_l --> center
  * outer    k_p1 --> center
  * right    k_u --> center
Y Axis (periodic):
  * center   j --> left
  * left     j_g --> center

In [15]:
# coriolis term
lat = grd['YC']
omega = 7.3/100000
f_ij = 2*omega*np.sin(np.deg2rad(lat))
f_ij

<xarray.DataArray 'YC' (face: 13, j: 4320, i: 4320)>
dask.array<mul, shape=(13, 4320, 4320), dtype=float32, chunksize=(1, 4320, 4320), chunktype=numpy.ndarray>
Coordinates:
    CS       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    Depth    (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    SN       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    XC       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    YC       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319

___________
## Term 2: Coriolis terms

### f*v

In [16]:
dij=4

for face in range(ds['face'].size):

    fv = ds['SSV_rotation'].isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))*f_ij.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) 
    fv = np.real(fv).rename('fv')
    fv = fv.chunk({'time': 1,'i':1080,'j':1080})
    fv.to_dataset().to_zarr(E_dir+'fv_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


### f*u

In [17]:
for face in range(ds['face'].size):
#for face in [1]:

    fu = ds['SSU_rotation'].isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))*f_ij.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    fu = np.real(fu).rename('fu')
    fu = fu.chunk({'time': 1,'i':1080,'j':1080})
    fu.to_dataset().to_zarr(E_dir+'fu_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


___________
## Term 2.2: zeta*uv

In [18]:
D = [xr.open_zarr(E_dir+'zeta_rotation_f%02d.zarr'%(face)) for face in range(face_all)]
ds_zeta = xr.concat(D, dim='face')
face_13 = range(0,13)
ds_zeta = ds_zeta.assign_coords(face=face_13) 
ds_zeta

<xarray.Dataset>
Dimensions:  (face: 13, i: 1080, j: 1080, time: 240)
Coordinates:
  * i        (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * j        (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    zeta     (face, j, i, time) float32 dask.array<chunksize=(1, 1080, 1080, 1), meta=np.ndarray>

In [19]:
dij=4

for face in range(ds['face'].size):

    zeta_v = ds_zeta['zeta'].isel(face=face)*ds['SSV_rotation'].isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    zeta_v = np.real(zeta_v).rename('zeta_v')
    zeta_v = zeta_v.chunk({'time': 1,'i':1080,'j':1080})
    zeta_v.to_dataset().to_zarr(E_dir+'zeta_v_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


In [20]:
dij=4

for face in range(ds['face'].size):

    zeta_u = ds_zeta['zeta'].isel(face=face)*ds['SSU_rotation'].isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    zeta_u = np.real(zeta_u).rename('zeta_u')
    zeta_u = zeta_u.chunk({'time': 1,'i':1080,'j':1080})
    zeta_u.to_dataset().to_zarr(E_dir+'zeta_u_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


________
# Term 3: Gradient of Bernoulli function

In [21]:
for face in range(ds['face'].size):
#for face in [1]:

    KE = (ds['SSU_rotation'].isel(face=face)**2 + ds['SSV_rotation'].isel(face=face)**2)/2 # KE
    # B represents KE here    
    dB_dx_raw = grid.diff(KE,'X')/ds.dxC.isel(face=face)     # d(KE)/dx
    dB_dy_raw = grid.diff(KE,'Y')/ds.dyC.isel(face=face)     # d(KE)/dy
    # i,j
    dB_dx_raw_ij = grid.interp(dB_dx_raw,'X')
    dB_dy_raw_ij = grid.interp(dB_dy_raw,'Y')
    # rotation 
    dB_dx =  dB_dx_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.CS.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) - dB_dy_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.SN.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    dB_dy =  dB_dx_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.SN.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) + dB_dy_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.CS.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))

    # store
    dB_dx = np.real(dB_dx).rename('dKE_dx')    
    dB_dx = dB_dx.chunk({'time': 1,'i':1080,'j':1080})
    dB_dx.to_dataset().to_zarr(E_dir+'dKE_dx_f%02d.zarr'% (face), mode='w')
    
    dB_dy = np.real(dB_dy).rename('dKE_dy')    
    dB_dy = dB_dy.chunk({'time': 1,'i':1080,'j':1080})
    dB_dy.to_dataset().to_zarr(E_dir+'dKE_dy_f%02d.zarr'% (face), mode='w')        

    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


In [22]:
# 240 time_length about 15 mins in total

for face in range(ds['face'].size):
#for face in [1]:
    
    g = 9.81

    dB_dx_raw = grid.diff(g*ds['Eta'].isel(face=face),'X')/ds.dxC.isel(face=face)     
    dB_dy_raw = grid.diff(g*ds['Eta'].isel(face=face),'Y') /ds.dyC.isel(face=face)     
    # i,j
    dB_dx_raw_ij = grid.interp(dB_dx_raw,'X')
    dB_dy_raw_ij = grid.interp(dB_dy_raw,'Y')
    # rotation
    dB_dx =  dB_dx_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.CS.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) - dB_dy_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.SN.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    dB_dy =  dB_dx_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.SN.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij)) + dB_dy_raw_ij.isel(i=slice(0,None,dij), j=slice(0,None,dij))*ds.CS.isel(face=face,i=slice(0,None,dij), j=slice(0,None,dij))
    
    # store
    dB_dx = np.real(dB_dx).rename('deta_dx')    
    dB_dx = dB_dx.chunk({'time': 1,'i':1080,'j':1080})
    dB_dx.to_dataset().to_zarr(E_dir+'deta_dx_f%02d.zarr'% (face), mode='w')

    dB_dy = np.real(dB_dy).rename('deta_dy')    
    dB_dy = dB_dy.chunk({'time': 1,'i':1080,'j':1080})
    dB_dy.to_dataset().to_zarr(E_dir+'deta_dy_f%02d.zarr'% (face), mode='w')        

    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


# Term 5: residuals

In [23]:
for face in range(13):
        
    ds_du_dt = xr.open_zarr(E_dir+'du_dt_f%02d.zarr'%face)
    ds_fv = xr.open_zarr(E_dir+'fv_f%02d.zarr'%face)
    ds_zv = xr.open_zarr(E_dir+'zeta_v_f%02d.zarr'%face)
    ds_dKE_dx = xr.open_zarr(E_dir+'dKE_dx_f%02d.zarr'%face)
    ds_deta_dx = xr.open_zarr(E_dir+'deta_dx_f%02d.zarr'%face)
    
    residuals_u = ds_du_dt['du_dt'] - ds_fv['fv'] - ds_zv['zeta_v'] + ds_dKE_dx['dKE_dx'] + ds_deta_dx['deta_dx']
    residuals_u = np.real(residuals_u).rename('residuals_u')
    residuals_u = residuals_u.chunk({'time': 1,'i':1080,'j':1080})
    residuals_u.to_dataset().to_zarr(E_dir+'residuals_u_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


In [24]:
for face in range(13):
        
    ds_dv_dt = xr.open_zarr(E_dir+'dv_dt_f%02d.zarr'%face)
    ds_fu = xr.open_zarr(E_dir+'fu_f%02d.zarr'%face)
    ds_zu = xr.open_zarr(E_dir+'zeta_u_f%02d.zarr'%face)
    ds_dKE_dy = xr.open_zarr(E_dir+'dKE_dy_f%02d.zarr'%face)
    ds_deta_dy = xr.open_zarr(E_dir+'deta_dy_f%02d.zarr'%face)
    
    residuals_v = ds_dv_dt['dv_dt'] + ds_fu['fu'] + ds_zu['zeta_u'] + ds_dKE_dy['dKE_dy'] + ds_deta_dy['deta_dy'] 
    residuals_v = np.real(residuals_v).rename('residuals_v')
    residuals_v = residuals_v.chunk({'time': 1,'i':1080,'j':1080})
    residuals_v.to_dataset().to_zarr(E_dir+'residuals_v_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))

face=0 - allready processed
face=1 - allready processed
face=2 - allready processed
face=3 - allready processed
face=4 - allready processed
face=5 - allready processed
face=6 - allready processed
face=7 - allready processed
face=8 - allready processed
face=9 - allready processed
face=10 - allready processed
face=11 - allready processed
face=12 - allready processed


In [ ]:
cluster.close()